In [212]:
import twitter
import tweepy
from tweepy import API
from tweepy import Cursor
import numpy as np
from sqlalchemy import create_engine
import pandas as pd
import time
from time import sleep
import os

ck = os.environ.get('CK')
cs = os.environ.get('CS')
atk = os.environ.get('ATK')
ats = os.environ.get('ATS')

# print(os.environ.get(ck))

auth = tweepy.OAuthHandler(ck, cs)
auth.set_access_token(atk, ats)
auth_api = API(auth)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [14]:
# Get a LIST of friends that you are following
list_of_friends_id = api.friends_ids()
print(len(list_of_friends_id))
list_of_friends_id[:5]

58


[15754281, 788524, 2768501, 16834659, 14138785]

In [222]:
# To get the ids of people that are following you
x = api.followers_ids()
print(len(x))
x[:5]

38


[17116782,
 1129863867960635396,
 264924050,
 731732329572376576,
 994672306013261824]

In [39]:
# Getting informations from other users timelines
y = api.user_timeline(15754281,count=5)
print(type(y),len(y))

<class 'tweepy.models.ResultSet'> 5


In [43]:
print(y[0].geo,y[0].id,y[0].favorite_count,y[0].retweet_count,y[0].lang)

None 1134764021163409408 25 3 en


In [17]:
# my id
my_id = api.me().id
len(api.user_timeline(my_id))

20

In [44]:
# Getting my id and taking info from it
my_id = api.me().id
y = api.user_timeline(my_id)
my_tweets_list = [i.id for i in y]

In [186]:
list_of_friends_id = api.friends_ids()

def theupdate_at_work():
    while True:
        df = pd.DataFrame(columns=np.arange(8))
        for user in list_of_friends_id:
            list_tweet = api.user_timeline(user,count=3,exclude_replies=True)
            for tweet in list_tweet:
                df.loc[len(df)] = [tweet.id,tweet.geo,tweet.favorite_count,tweet.retweet_count,tweet.lang,
                                user,tweet._json['user']['id'],tweet._json['text']]
                                   #tweet._json['entities']['urls'][0]['url']]
        df[8]=df[2]+df[3]*2   #scoring of the tweets
        df = df[df[4]=='en']
        df[9] = [False if i[:2]=='RT' else True for i in df[7]]
        df = df.sort_values(by=9,ascending=False)
        df = df[df[9]]
        counter = 0
        counter_tweet = 0
        counter_errors = 0
        for tw in list(df[0]):
            # counter+=1
            # print('Global counter: ',counter, end='\r')
            if counter_tweet==5:
                break
            else:
                try:
                    api.retweet(tw)
                    counter_tweet+=1
                    print('tweet number: ',counter_tweet,end='\r')
                    for second in range(180):
                        time.sleep(1)
                        print('timer: ',179-second,end='\r')
                except:
                    pass

In [187]:
theupdate_at_work()

In [179]:
tweet = api.user_timeline(15754281,count=3,exclude_replies=True)

In [183]:
tweet[0]._json['entities']['urls']

[{'url': 'https://t.co/7RP5YhaiBl',
  'expanded_url': 'https://twitter.com/i/web/status/1134779129126109185',
  'display_url': 'twitter.com/i/web/status/1…',
  'indices': [117, 140]}]

In [ ]:
tweet._json['entities']['urls'][0]['url']